In [ ]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go
from matplotlib.ticker import FormatStrFormatter
import datatable as dt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import optuna


import warnings
warnings.filterwarnings('ignore')

for dirname, _, filenames in os.walk(''):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = pd.read_csv(r'train.csv', index_col='id', nrows=1000000)
test_df = pd.read_csv(r'test.csv', index_col='id', nrows=500000)
submission_df= pd.read_csv(r'sample_submission.csv', index_col='id', nrows=500000)

In [ ]:
print(train_df.shape)
print(test_df.shape)


In [ ]:
train_df.describe().T

In [ ]:
#Lets find traget variable distribution
target_varibale=train_df['target']

In [ ]:
import seaborn as sns
pal = ['#5ddef4','#8e99f3']
plt=sns.countplot(x=train_df.target,palette=pal)
plt.set_title('Target  distribution', fontsize=20, y=1.05)

In [ ]:
display(train_df.isna().sum().sum())
display(test_df.isna().sum().sum())

In [ ]:
categorical_features=[]
numerical_features=[]
for i in train_df.columns:
    if train_df[i].dtype=='float64':
#         print('yes')
        numerical_features.append(i)
    elif i!='target':
        categorical_features.append(i)
#         print('no')

In [ ]:
display(len(categorical_features))
display(len(numerical_features))

In [ ]:
df_num_features = pd.concat([train_df[numerical_features], train_df['target']], axis=1)

In [ ]:
import matplotlib.pyplot as plt
df_cat = pd.concat([train_df[categorical_features], train_df['target']], axis=1)
fig, ax = plt.subplots(1, 1, figsize=(16 , 16))
corr = df_cat.sample(10000, random_state=2021).corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(corr, ax=ax, square=True, center=0, linewidth=1,
        cmap=sns.diverging_palette(0,255,sep=77, as_cmap=True),vmax=0.5, vmin=-0.5,
        cbar_kws={"shrink": .85}, mask=mask )
ax.set_title('Correlation heatmap: Categorical features', fontsize=24, y= 1.05)
plt.show()

In [ ]:
!pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_validate
import warnings 
warnings.filterwarnings('ignore')


df_train = pd.read_csv( "train.csv",
    index_col='id',

)

X_test = pd.read_csv( "test.csv", index_col='id')

FEATURES = df_train.columns[:-1]
TARGET = df_train.columns[-1]

X = df_train.loc[:, FEATURES]
y = df_train.loc[:, TARGET]

seed = 0
fold = 5


In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 9500,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.6000000000000001,
    'min_child_weight': 56.41980735551558,
    'reg_lambda': 75.56651890088857,
    'reg_alpha': 0.11766857055687065,
    'gamma': 0.6407823221122686,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
    }

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(max_depth=6,
    subsample=.7,
    colsample_bytree=0.2,
    colsample_bylevel=0.6,
    min_child_weight=50,reg_lambda=75,reg_alpha=0.11,gamma=0.6,booster='gbtree',use_label_encoder=False,
    n_jobs=-1,
    tree_method='gpu_hist',
    sampling_method='gradient_based', 
    random_state= seed,
)
def score(X, y, model_xgb, cv):
    scoring = ["roc_auc"]
    scores = cross_validate(
        model_xgb, X, y, scoring=scoring, cv=cv, return_train_score=True
    )
    scores = pd.DataFrame(scores).T
    return scores.assign(
        mean = lambda x: x.mean(axis=1),
        std = lambda x: x.std(axis=1),
    )

scores = score(X, y, model_xgb, cv=fold)
display(scores)

In [ ]:
!pip install xgboost

In [ ]:
from catboost import Pool, CatBoostClassifier, cv, CatBoostRegressor

model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           l2_leaf_reg=3.5, 
                           depth=8, rsm=0.98,
                           loss_function= 'Logloss',
                           eval_metric='AUC',use_best_model=True,
                           random_seed=42)



In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=0.99,random_state=1236)

In [ ]:

cate_features_index = np.where(X.dtypes != float)[0]

In [ ]:
X_test = pd.read_csv("test.csv", index_col='id')
y_pred_xgb = pd.Series(
    joblib_LR_model.predict_proba(X_test)[:, 1],
    index=X_test.index,
    name=TARGET,
)
y_pred_xgb.to_csv("itsokay.csv")


In [1]:
import pandas as pd
import numpy as np
import datatable as dt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import optuna

In [ ]:

train = dt.fread('train.csv').to_pandas().drop('id', axis=1)

test = dt.fread('test.csv').to_pandas().drop('id', axis=1)

In [ ]:
X = train.drop('target', axis=1).copy()
y = train['target'].copy()
X_test = test.copy()

del train
del test

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 4000,
    'subsample': 0.72,
    'colsample_bytree': 0.21,
    'colsample_bylevel': 0.6,
    'min_child_weight': 57,
    'reg_lambda': 75,
    'reg_alpha': 0.12,
    'gamma': 0.65,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
    }

In [ ]:
%%time
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=786)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    model3 = XGBClassifier(**params)
    
    model3.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=True)
    
    pred_valid = model3.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    test_preds = model3.predict_proba(X_test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

In [ ]:
ss = dt.fread('sample_submission.csv').to_pandas()
predictions = np.mean(np.column_stack(preds),axis=1)

ss['target'] = predictions
ss.to_csv('xgb1111.csv', index=False)
ss.head()